<a href="https://colab.research.google.com/github/softmurata/AudioML/blob/main/avatar/chatavatar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!pip install -q gradio
!pip install -q sentence_transformers

In [2]:
# ユースケース1: 新規顧客獲得
case1 = [
    {"Q": "営業: こんにちは、村田さん。弊社のサービスに興味を持っていただきありがとうございます。まず最初に、お手伝いできることや特にお困りごとはありますか？",
     "A": "お客様: はい、最近村田さんに課題を抱えていて、それに対する解決策を模索しています。"},
    {"Q": "営業: それは理解できます。お聞かせいただければと思いますが、現在の課題の具体的なポイントは何でしょうか？",
     "A": "お客様: 村田さんが効率的に機能していないことが主な課題です。"},
    {"Q": "営業: なるほど、それは重要なポイントですね。弊社のサービスは、村田さんの効率向上に特に焦点を当てており、その点でお手伝いできるかもしれません。どのような要件や期待をお持ちでしょうか？",
     "A": "お客様: 具体的な要件としては、村田さんがスムーズに行えることと、村田さんの改善案があれば教えていただきたいです。"}
]

# ユースケース2: 既存顧客へのアップセル
case2 = [
    {"Q": "営業: お世話になっております、村田さん。これまでのご利用ありがとうございます。今回は、新たな機能やアップグレードについてお知らせがあります。お時間いただけますでしょうか？",
     "A": "お客様: はい、どのような内容ですか？"},
    {"Q": "営業: 弊社では最近、〇〇機能を追加しました。これにより、〇〇の利便性が向上し、業務の効率化が期待できます。ご興味がありましたら、詳細をご案内させていただけますか？",
     "A": "お客様: 興味がありますね。具体的にどのような変更があるのでしょうか？"},
    {"Q": "営業: それでは、〇〇機能の詳細や導入メリットについてお話しいたしますね。",
     "A": "お客様: 聞きたいことがあれば質問させてください。"}
]

# ユースケース3: フィードバック収集
case3 = [
    {"Q": "営業: こんにちは、〇〇さん。ご愛顧いただきありがとうございます。今日は、弊社のサービスに関するフィードバックをお伺いしたく存じます。ご意見やご感想がありましたらお聞かせいただけますか？",
     "A": "お客様: あまり大きな問題はないですが、時々〇〇に関する使い勝手が気になります。"},
    {"Q": "営業: それは貴重なご意見ですね。具体的にどのような点が気になりましたか？",
     "A": "お客様: 〇〇の操作が少し複雑で、もっと直感的になればいいなと思います。"},
    {"Q": "営業: 了解しました。おっしゃる通り、使いやすさが重要ですね。今後の開発で改善できるように努めてまいります。他にも何かありましたらお気軽におっしゃってください。",
     "A": "お客様: ありがとうございます。他には特に問題ないですが、改善点があれば随時お知らせいたします。"}
]

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import glob

def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)

    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return 0
    else:
        return dot_product / (magnitude_v1 * magnitude_v2)


model = SentenceTransformer('pkshatech/GLuCoSE-base-ja')

questions = []
for item in case1:
  questions.append(item["Q"])
question_embeddings = model.encode(questions)

videos = sorted(glob.glob("/content/case1/*.mp4"))

In [ ]:
# 音声入力ー＞文字起こしみたいなところもできるようにしておくといい？

In [ ]:
import gradio as gr
import cv2

blocks = gr.Blocks()

message_history = []

with blocks as demo:
  with gr.Row():
    gr.Markdown("## 営業トーク")
  with gr.Column():
    with gr.Row():
      video = gr.Video(label="Face Video", autoplay=True)
      chatbot = gr.Chatbot()
    with gr.Row():
      input = gr.Textbox(label="input text", placeholder="input your text...")
      send_button = gr.Button("Send")

    def video_identify(query):
      global message_history

      message_history.append({
          "role": "user",
          "content": query
      })

      source_embeddings = model.encode([query])

      cos_sim_value_list = []
      for t in question_embeddings:
        cos_sim_value = cosine_similarity(t, source_embeddings.reshape(-1))
        cos_sim_value_list.append(cos_sim_value)

      top_id = np.argsort(cos_sim_value_list)[::-1][0]

      answer = case1[top_id]["A"]
      video_path = videos[top_id]

      message_history.append({
          "role": "assistant",
          "content": answer
      })

      return video_path

    def chat_output():

      return [(message_history[i]["content"], message_history[i + 1]["content"]) for i in range(0, len(message_history) - 1, 2)]


    video.play(
        chat_output,
        inputs=[],
        outputs=[chatbot]
    )


    send_button.click(
        video_identify,
        inputs=[input],
        outputs=[video]
    )



demo.launch(debug=True)

In [ ]:
# case1
case1 = [
    {"Q": "営業: こんにちは、村田さん。弊社のサービスに興味を持っていただきありがとうございます。まず最初に、お手伝いできることや特にお困りごとはありますか？",
     "A": "お客様: はい、最近村田さんに課題を抱えていて、それに対する解決策を模索しています。"},
    {"Q": "営業: それは理解できます。お聞かせいただければと思いますが、現在の課題の具体的なポイントは何でしょうか？",
     "A": "お客様: 村田さんが効率的に機能していないことが主な課題です。"},
    {"Q": "営業: なるほど、それは重要なポイントですね。弊社のサービスは、村田さんの効率向上に特に焦点を当てており、その点でお手伝いできるかもしれません。どのような要件や期待をお持ちでしょうか？",
     "A": "お客様: 具体的な要件としては、村田さんがスムーズに行えることと、村田さんの改善案があれば教えていただきたいです。"}
]

音声入力デモ

In [ ]:
import torch
import gradio as gr
import time
import yt_dlp as youtube_dl
from transformers import pipeline
from transformers.pipelines.audio_utils import ffmpeg_read

import tempfile
import os

MODEL_NAME = "openai/whisper-large-v3"
BATCH_SIZE = 8
FILE_LIMIT_MB = 1000
YT_LENGTH_LIMIT_S = 3600  # limit to 1 hour YouTube files

device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

def transcribe(inputs, task):
    if inputs is None:
        raise gr.Error("No audio file submitted! Please upload or record an audio file before submitting your request.")

    text = pipe(inputs, batch_size=BATCH_SIZE, generate_kwargs={"task": task}, return_timestamps=True)["text"]
    return  text


def _return_yt_html_embed(yt_url):
    video_id = yt_url.split("?v=")[-1]
    HTML_str = (
        f'<center> <iframe width="500" height="320" src="https://www.youtube.com/embed/{video_id}"> </iframe>'
        " </center>"
    )
    return HTML_str

def download_yt_audio(yt_url, filename):
    info_loader = youtube_dl.YoutubeDL()

    try:
        info = info_loader.extract_info(yt_url, download=False)
    except youtube_dl.utils.DownloadError as err:
        raise gr.Error(str(err))

    file_length = info["duration_string"]
    file_h_m_s = file_length.split(":")
    file_h_m_s = [int(sub_length) for sub_length in file_h_m_s]

    if len(file_h_m_s) == 1:
        file_h_m_s.insert(0, 0)
    if len(file_h_m_s) == 2:
        file_h_m_s.insert(0, 0)
    file_length_s = file_h_m_s[0] * 3600 + file_h_m_s[1] * 60 + file_h_m_s[2]

    if file_length_s > YT_LENGTH_LIMIT_S:
        yt_length_limit_hms = time.strftime("%HH:%MM:%SS", time.gmtime(YT_LENGTH_LIMIT_S))
        file_length_hms = time.strftime("%HH:%MM:%SS", time.gmtime(file_length_s))
        raise gr.Error(f"Maximum YouTube length is {yt_length_limit_hms}, got {file_length_hms} YouTube video.")

    ydl_opts = {"outtmpl": filename, "format": "worstvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best"}

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([yt_url])
        except youtube_dl.utils.ExtractorError as err:
            raise gr.Error(str(err))


def yt_transcribe(yt_url, task, max_filesize=75.0):
    html_embed_str = _return_yt_html_embed(yt_url)

    with tempfile.TemporaryDirectory() as tmpdirname:
        filepath = os.path.join(tmpdirname, "video.mp4")
        download_yt_audio(yt_url, filepath)
        with open(filepath, "rb") as f:
            inputs = f.read()

    inputs = ffmpeg_read(inputs, pipe.feature_extractor.sampling_rate)
    inputs = {"array": inputs, "sampling_rate": pipe.feature_extractor.sampling_rate}

    text = pipe(inputs, batch_size=BATCH_SIZE, generate_kwargs={"task": task}, return_timestamps=True)["text"]

    return html_embed_str, text

In [ ]:
import gradio as gr

blocks = gr.Blocks()

with blocks as demo:
  with gr.Row():
    with gr.Column():
      input_audio = gr.Audio(label="Input Audio", sources=["microphone", "upload"], type="filepath")
      send_button = gr.Button("Send")
  with gr.Row():
    output = gr.Textbox(label="Output")

  def audio_func(inputs):
    if inputs is None:
        raise gr.Error("No audio file submitted! Please upload or record an audio file before submitting your request.")

    # task = "transcribe"
    # text = pipe(inputs, batch_size=BATCH_SIZE, generate_kwargs={"task": task}, return_timestamps=True)["text"]
    text = "hello output"

    return text

  send_button.click(
      audio_func,
      inputs=[input_audio],
      outputs=[output]
  )

demo.launch()




